In [ ]:
# pip install xlsxwriter      # writing excel files

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#### PREPARATION OF DATASETS FOR THE MODEL

### RELATIVE_PROTEOMICS.TXT
# I created the datasets using Excel and then just saved necessary columns for different conditions.
# 1. I copied in a new excel sheet from MaxQuant excel output "proteinGroups" uniprot IDs, Gene names, molecular weight, and intensities.
# 2. Calculated molar abundances of each protein in the dataset using total protein content and mean average of intensities per condition per sample.

# Note! the f factor for enhanceGEM pipeline doesn't change with different Ptot for the same proteomics dataset.
# It is a proportionality coefficient that is used with the provided Ptot (in getModelParameters) for constraining pool.

# 3. Saved output using following lines:
df1=pd.read_excel("proteins_rhto_v02.xlsx", sheet_name='proteomics', usecols='B,AH') # B corresponds to Gene names, other column represents molar abundances
df1

In [ ]:
df1['mmol_gDCW_average'] = df1['mmol_gDCW_average'].replace(np.nan, 0)
df1

In [ ]:
df1.to_csv('relative_proteomics.txt', sep='\t', index=False)
df1

In [ ]:
### ABS_PROTEOMICS.TXT

# Followed the same guidelines as for relative_proteomics.txt, the only difference being that here all replicates per condition have to be included.
# It is because GECKO includes the filtering of absolute proteomics measurements.

# Saved output:
df2=pd.read_excel("proteins_rhto_v02.xlsx", sheet_name='proteomics', usecols='A,B,CO,CP')
df2.to_csv('abs_proteomics.txt', sep='\t', index=False)
df2

In [ ]:
### FERMENTATIONDATA.TXT: to constrain the model(s) during generate_protModels pipeline
#df=pd.read_csv("fermentationData.txt", sep='\t')
df3=pd.read_excel("proteins_rhto_v02.xlsx", sheet_name='fermentationData', skiprows=[0,1,2,4,6,7,8], usecols='A,B,C,D,E,F,G')
df3.to_csv('fermentationData.txt', sep='\t', index=False)
df3

In [ ]:
### CHEMOSTATDATA.TSV: to fit growth-associated maintenance (GAM) during generate_protModels pipeline
#df=pd.read_csv('chemostatData.tsv',sep='\t')
#manually edit the file usign excel
df4=pd.read_excel('proteins_rhto_v02.xlsx', sheet_name="chemostatData", skiprows=[0,1,2,6,7,8])
df4.to_csv('chemostatData.tsv', sep='\t', index=False)
df4

In [ ]:
### selectedAnnotation
#df5=pd.read_excel("proteins_rhto.xlsx", sheet_name='selectedAnnotation', usecols='A,D')
df5=pd.read_excel("proteins_rhto.xlsx", sheet_name='selectedAnnotation')
df5.to_csv('selectedAnnotation.txt', sep='\t', index=False)
df5

In [ ]:
df55=pd.read_excel("proteins_rhto.xlsx", sheet_name='uniprot_swissprot_full_10k', usecols='A,C')
df55

In [ ]:
output=pd.merge(df5, df55, on='uniprot')
output

In [ ]:
### ribosome
#df1=pd.read_excel("proteins_rhto.xlsx", sheet_name='ribosomal_2', usecols='A,C,D')
#df2=pd.read_excel("ribosome_full.xlsx", sheet_name='Sheet1', usecols='A,E,F,G')
#output=pd.merge(df1, df2, on='Entry')
output=pd.read_excel("proteins_rhto.xlsx", sheet_name='ribosome')
output.to_csv('ribosome.txt', sep='\t', index=False)
output

In [ ]:
### UNIPROT.TAB: to assign enzymatic constraints during the enhanceGEM pipeline
## 2.Generate a new, corrected uniprot.tab file with more ec numbers to increase the coverage of enzymes in ecModel
# 2.1.Create excel file from old uniprot.tab file
df6=pd.read_csv("uniprot_original.tab", sep='\t')
# 2.2.Manually combine in excel ec numbers from rhtoModel with ec numbers from uniprot.tab, based on gene association
# 2.3.Create new uniprot file:
df6=pd.read_excel("uniprot_rhto_corrected.xlsx", sheet_name='uniprot_manually_with_new_ec')
df6.to_csv('uniprot.tab', sep='\t', index=False)

In [ ]:
df1=pd.read_excel("uniprot_rhto_corrected.xlsx", sheet_name='uniprot_original', usecols='A,B,C,D,E')
df2=pd.read_excel("uniprot_rhto_corrected.xlsx", sheet_name='new_with_ec')
df2.to_csv('uniprot_new_EC.tab', sep='\t', index=False)

In [ ]:
### PROTEINS.TXT: protein pool in geckopy               # not used, as geckopy offers only proteomics integration
# 1.1.Prepare datasets:
# a) protein IDs, proteomics measurements in intensities;
# b) uniprot file with protein IDs and molecular weights for R.toruloides
# 1.2.Merge based on protein IDs for mw and abundance
df1=pd.read_excel("proteins_rhto.xlsx", sheet_name='intensities_to_fractions', usecols='A,D')
df2=pd.read_excel("proteins_rhto.xlsx", sheet_name='uniprot_mws_g_mol')
proteins_fractions=pd.merge(df1, df2[['uniprot', 'mw']], on='uniprot')
# 1.3.Create excel sheet "proteins_fractions", using code below (xlsxwriter)
df=pd.read_excel("proteins_rhto.xlsx", sheet_name='proteins_fractions', usecols='A,B,C')
df.to_csv('proteins.txt',index=False)
# 1.4.Move proteins.txt to geckopy/data_files of your computer

In [ ]:
### OTHERS
df1=pd.read_csv("manual_data.txt", sep='\t')
df1

In [ ]:
#F-factor calculations:
df1=pd.read_excel("rhto_f_factor.xlsx", sheet_name='f_factor_ecModelP', usecols='J')
df2=pd.read_excel("rhto_f_factor.xlsx", sheet_name='f_factor_ecModelP_calculations', usecols='A,E')
f_factor=pd.merge(df1, df2[['uniprotID', 'REF_048_g/gDW_mean']], on='uniprotID')
f_factor

In [ ]:
uniprot = pd.read_csv("uniprot.tab", sep='\t')
uniprot

In [ ]:
#How many enzymatic proteins are covered by proteomics dataset?
df1=pd.read_excel("proteins_rhto.xlsx", sheet_name='uniprot_full_10k', usecols='A')
df2=pd.read_excel("proteins_rhto.xlsx", sheet_name='proteomics_mmol_gDW', usecols='A')
output=pd.merge(df1, df2)
output
# No, it doesn't work like that because uniprot contains all proteins, and sorting according to ec numbers also not possible, because many lack them!

In [ ]:
#How many enzymatic proteins were filtered out to make v5 ecModelP?
df1=pd.read_excel("proteins_rhto.xlsx", sheet_name='abs_proteomics_calculations', usecols='A,B,AZ,BA,BB')
df2=pd.read_excel("proteins_rhto.xlsx", sheet_name='v5_pool_match')
output=pd.merge(df1, df2, on='uniprotID')
output

In [ ]:





# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
output.to_excel(writer, sheet_name='Sheet1', index=None)

# Close the Pandas Excel writer and output the Excel file.
writer.save()






In [ ]:
#### RESULT PREPARATION


In [ ]:
#Save result files for escher
df1=pd.read_excel("results_rhto.xlsx", sheet_name='fluxes', usecols='AG,AH')
df1.to_csv('fluxes_XNlim.csv',index=False)

In [ ]:

df1=pd.read_excel("results_rhto.xlsx", sheet_name='enzUsage', usecols='AX,AY')
df1.to_csv('capUse_XNlim.csv',index=False)

In [ ]:
#Assign selected annotation to enzyme usage reports
#df1=pd.read_excel("results_rhto.xlsx", sheet_name='assign_enzNames')
#df2=pd.read_excel("enzUsage.xlsx", sheet_name='Xexp_cobra')
#output=pd.concat([df1, df2], axis=1)
#output

In [ ]:
#Assign enzyme names to usage reports
df1=pd.read_excel("results_rhto.xlsx", sheet_name='assign_enzNames')
df2=pd.read_excel("proteins_rhto.xlsx", sheet_name='uniprot_swissprot_full_10k', usecols='A,B')
output=pd.merge(df1, df2, on='uniprot')
output
#output.to_csv('capUsage_annotated.txt', sep='\t', index=False)